In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 

In [ ]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_train.head()

In [ ]:
df_train = df_train.drop(['Ticket','Name','PassengerId','Cabin'],axis=1).fillna(method='ffill').fillna(method='bfill')
df_train.head()

In [ ]:
X_titanic_full = df_train.iloc[:,1:].values
Y_titanic_full = df_train.iloc[:,0].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_titanic_full, Y_titanic_full, test_size=0.20)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
X_train_onehot = enc.fit_transform(X_train)
X_test_onehot = enc.fit_transform(X_test)
X_titanic_full_onehot = enc.fit_transform(X_titanic_full)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

acc_train = [] 
acc_test = []
depth_interval = range(1,20)
for i in depth_interval:
    clf = RandomForestClassifier(n_estimators=100,max_depth=i)
    clf.fit(X_train_onehot, Y_train)
    Y_predicted = clf.predict(X_test_onehot)
    Y_predicted_train = clf.predict(X_train_onehot)
    acc_test.append(accuracy_score(Y_test, Y_predicted))
    acc_train.append(accuracy_score(Y_train, Y_predicted_train))

from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(depth_interval, acc_train, 'b', label='Train Accuracy')
line2, = plt.plot(depth_interval, acc_test, 'r', label='Test Accuracy')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('Accuracy score')
plt.xlabel('max depth')
plt.show()

In [ ]:
acc_train = [] 
acc_test = []
trees_interval = range(10,200,10)
for i in depth_interval:
    clf = RandomForestClassifier(n_estimators=i,max_depth=10)
    clf.fit(X_train_onehot, Y_train)
    Y_predicted = clf.predict(X_test_onehot)
    Y_predicted_train = clf.predict(X_train_onehot)
    acc_test.append(accuracy_score(Y_test, Y_predicted))
    acc_train.append(accuracy_score(Y_train, Y_predicted_train))

from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(trees_interval, acc_train, 'b', label='Train Accuracy')
line2, = plt.plot(trees_interval, acc_test, 'r', label='Test Accuracy')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('Accuracy score')
plt.xlabel('Trees number')
plt.show()

In [ ]:
clf = RandomForestClassifier(n_estimators=100,max_depth=10,criterion='gini')
clf.fit(X_train_onehot, Y_train)
Y_predicted = clf.predict(X_test_onehot)
Y_predicted_train = clf.predict(X_train_onehot)
print(classification_report(Y_test, Y_predicted))

In [ ]:
clf = RandomForestClassifier(n_estimators=100,max_depth=10,criterion='entropy')
clf.fit(X_train_onehot, Y_train)
Y_predicted = clf.predict(X_test_onehot)
Y_predicted_train = clf.predict(X_train_onehot)
print(classification_report(Y_test, Y_predicted))

**Parametes choosen are:** 
* Max depth = 10
* Number of estimators = 100
* Criterion = Entropy

In [ ]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=100,max_depth=5,criterion='entropy')
scores = cross_val_score(clf, X_titanic_full_onehot, Y_titanic_full, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
clf = RandomForestClassifier(n_estimators=100,max_depth=5,criterion='entropy')
clf.fit(X_titanic_full_onehot, Y_titanic_full)

df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
df_test = df_test.drop(['Ticket','Name','Cabin'],axis=1)
df_test = df_test.fillna(method='ffill').fillna(method='ffill').fillna(method='bfill')

df_submission = df_test[['PassengerId']]

df_test = df_test.drop(['PassengerId'],axis=1)

X_titanic_full_test = np.array(df_test.values)
X_titanic_full_test_ordinal = enc.fit_transform(X_titanic_full_test)

Y_titanic_full_test = clf.predict(X_titanic_full_test_ordinal)

df_submission['Survived'] = pd.Series(Y_titanic_full_test)
df_submission.to_csv('titanic_submission.csv',index=False)